# **Geometric Parametrization**
This lab focuses on performing a ROM on a parametric system with variable domain geometry.

In [ ]:
!git clone https://github.com/fvicini/CppToPython.git
%cd CppToPython

In [ ]:
!git submodule init
!git submodule update

In [ ]:
!mkdir -p externals
%cd externals
!cmake -DINSTALL_VTK=OFF -DINSTALL_LAPACK=OFF ../gedim/3rd_party_libraries
!make -j4
%cd ..

In [ ]:
!mkdir -p release
%cd release 
!cmake -DCMAKE_PREFIX_PATH="/content/CppToPython/externals/Main_Install/eigen3;/content/CppToPython/externals/Main_Install/triangle;/content/CppToPython/externals/Main_Install/tetgen;/content/CppToPython/externals/Main_Install/googletest" ../
!make -j4 GeDiM4Py
%cd ..

In [ ]:
import numpy as np
import GeDiM4Py as gedim
from scipy.sparse.linalg import splu
import time

In [ ]:
lib = gedim.ImportLibrary("./release/GeDiM4Py.so")

config = { 'GeometricTolerance': 1.0e-8 }
gedim.Initialize(config, lib)

## Poisson problem on variable geometry

Solve the following equation on the domain ${\tilde{\Omega}} = \tilde{\Omega}_1 \cup \tilde{\Omega}_2 =  (0, 1) \times (0, 1) \cup (1, 1+
\mu) \times (0, 1)$

$$
\begin{cases}
\nabla \cdot (\nabla u) = f & \text{in } \tilde{Ω}\\
u = 0 & \text{in } \partial \tilde{Ω}
\end{cases}
$$

The parametric space is $\mathcal P = [1, 3.5]$.

### Map to reference domain

**Goal**: build the ROM space using a reference domain

We choose as **reference domain** $\Omega$ the case $\mu = 1$.

Thus, $\Omega = \Omega_1 \cup \Omega_2 = [0,1] \times [0,1] \cup [1,2] \times [0,1]$.

The affine transformations are the following:
$$\tilde{\mathbf{x}} = \Phi_{\Omega_1}(\mathbf{x}, \mu) = \mathbb{I}\mathbf{x} + \mathbf{0}= \begin{bmatrix}
1 & 0 \\
0 & 1 
\end{bmatrix}\mathbf{x} + \begin{pmatrix}
0\\
0
\end{pmatrix} \quad \forall \tilde{\mathbf{x}} \in \tilde{\Omega_1}$$
$$\tilde{\mathbf{x}} = \Phi_{\Omega_2}(\mathbf{x}, \mu) = \mathbb{A}\mathbf{x} + \mathbf{c} = \begin{bmatrix}
\mu & 0 \\
0 & 1 
\end{bmatrix}\mathbf{x} + \begin{pmatrix}
1-\mu\\
0
\end{pmatrix} \quad \forall \tilde{\mathbf{x}} \in \tilde{\Omega_2}$$

Notice that
$$J_{\Phi_{\Omega_2}} =\begin{bmatrix}
\mu & 0 \\
0 & 1 
\end{bmatrix} ⇒ J^{-1}_{\Phi_{\Omega_2}} =\begin{bmatrix}
\frac{1}{\mu} & 0 \\
0 & 1 
\end{bmatrix}$$
and $|J_{\Phi_{\Omega_2}}| = \mu$

In [ ]:
def Map(points, mu):
  numPoints = points.shape[1]
  mappedPoints = np.copy(points)

  for p in range(1, numPoints):
    if (points[0, p] > 1.0 + 1.0e-8):
      mappedPoints[0, p] = mu * points[0, p] + (1. - mu)
  return mappedPoints

### Problem data

For this Lab we would like to find the solution
$$u = 16 x y (1 + \mu - x) (1-y)$$

Thus, the forcing term reads
$$f = 32 [x(1+\mu-x) + y(1-y)]$$

In [ ]:
def forcing_term(points):
  return 32.0 * (points[1,:] * (1.0 - points[1,:]) + points[0,:] * (1.0 + MU_TILDE - points[0,:]))
def exact_solution(points):
  return 16.0 * (points[1,:] * (1.0 - points[1,:]) * points[0,:] * (1.0 + MU_TILDE - points[0,:]))
def exact_derivative_solution(direction, points):
  if direction == 0:
    values = 16.0 * (1.0 + MU_TILDE - 2.0 * points[0,:]) * points[1,:] * (1.0 - points[1,:])
  elif direction == 1:
    values = 16.0 * (1.0 - 2.0 * points[1,:]) * points[0,:] * (1.0 + MU_TILDE - points[0,:])
  else:
    values = np.zeros(points.shape[1])
  return values

### Forcing Terms with map applied

The problem on the reference domain $Ω$ shall be computing applying the transformation function to the original problem.

Thus we will have:

$$\tilde{a}(\tilde{u}, \tilde{v}) = \int_{\tilde{\Omega}} \tilde{\nabla} \tilde{u}(\tilde{\mathbf{x}})\cdot \tilde{\nabla} \tilde{v}(\tilde{\mathbf{x}}) = \int_{\Omega = \Phi^{-1}(\tilde{\Omega})} \tilde{\nabla} \tilde{u}(\Phi(\mathbf{x})) \cdot \tilde{\nabla} \tilde{v}(\Phi(\mathbf{x}))|J_{\Phi}| ⇒$$
$$a(u, v) = \int_{\Omega = \Phi^{-1}(\tilde{\Omega})} [J_{\Phi}^{-1}\nabla u(\mathbf{x})] \cdot [J_{\Phi}^{-1}\nabla v(\mathbf{x})]|J_{\Phi}|$$
and:
$$\tilde{f}(\tilde{v}) = \int_{\tilde{\Omega}} \tilde{f}(\tilde{\mathbf{x}})\tilde{v}(\tilde{\mathbf{x}}) = \int_{\Omega = \Phi^{-1}(\tilde{\Omega})} \tilde{f}(\Phi(\mathbf{x})) \tilde{v}(\Phi(\mathbf{x}))|J_{\Phi}| ⇒$$
$$f(v) = \int_{\Omega = \Phi^{-1}(\tilde{\Omega})} f(\mathbf{x}) v(\mathbf{x})|J_{\Phi}|$$

**NB** $\tilde{f}$ and $f$ are NOT the same function, as $f = \tilde{f} \circ \Phi$ !!!

In [ ]:
def forcing_term_ref_11(points):
  return 32.0 * (points[1,:] * (1.0 - points[1,:]) + points[0,:] * (1.0 - points[0,:]))
def forcing_term_ref_12(points):
  return 32.0 * points[0,:]
def forcing_term_ref_21(points):
  return 32.0 * (points[1,:] * (1.0 - points[1,:]))
def forcing_term_ref_22(points):
  return 32.0 * (points[0,:] * (2.0 - points[0,:]))
def forcing_term_ref_23(points):
  return 32.0 * (2.0 - points[0,:])

### Function for assembling the Offline problem

In [ ]:
def OmegaTilde1(numPoints, points):
  matPoints = gedim.make_nd_matrix(points, (3, numPoints), np.double)
  values = np.zeros(numPoints)
  for p in range(0, numPoints):
    if (matPoints[0,p] <= (1.0 + 1.0e-8)):
      values[p] = 1.
  return values.ctypes.data
def OmegaTilde2_1(numPoints, points):
  matPoints = gedim.make_nd_matrix(points, (3, numPoints), np.double)
  values = np.zeros((3, numPoints), order='F')
  for p in range(0, numPoints):
    if (matPoints[0,p] > (1.0 + 1.0e-8)):
      values[0, p] = 1.
  return values.ctypes.data
def OmegaTilde2_2(numPoints, points):
  matPoints = gedim.make_nd_matrix(points, (3, numPoints), np.double)
  values = np.zeros((3, numPoints), order='F')
  for p in range(0, numPoints):
    if (matPoints[0,p] > (1.0 + 1.0e-8)):
      values[2, p] = 1.
  return values.ctypes.data

def ForcingTerm(numPoints, points):
  matPoints = gedim.make_nd_matrix(points, (3, numPoints), np.double)
  values = forcing_term(matPoints)
  return values.ctypes.data
def Poisson_exactSolution(numPoints, points):
  matPoints = gedim.make_nd_matrix(points, (3, numPoints), np.double)
  values = exact_solution(matPoints)
  return values.ctypes.data
def Poisson_exactDerivativeSolution(direction, numPoints, points):
  matPoints = gedim.make_nd_matrix(points, (3, numPoints), np.double)
  values = exact_derivative_solution(direction, matPoints)
  return values.ctypes.data

def ForcingTerm11(numPoints, points):
  matPoints = gedim.make_nd_matrix(points, (3, numPoints), np.double)
  values = forcing_term_ref_11(matPoints)
  for p in range(0, numPoints):
    if (matPoints[0, p] > (1.0 + 1.0e-8)):
      values[p] = 0. 
  return values.ctypes.data
def ForcingTerm12(numPoints, points):
  matPoints = gedim.make_nd_matrix(points, (3, numPoints), np.double)
  values = forcing_term_ref_12(matPoints)
  for p in range(0, numPoints):
    if (matPoints[0, p] > (1.0 + 1.0e-8)):
      values[p] = 0. 
  return values.ctypes.data

def ForcingTerm21(numPoints, points):
  matPoints = gedim.make_nd_matrix(points, (3, numPoints), np.double)
  values = forcing_term_ref_21(matPoints)
  for p in range(0, numPoints):
    if (matPoints[0, p] <= (1.0 + 1.0e-8)):
      values[p] = 0. 
  return values.ctypes.data
def ForcingTerm22(numPoints, points):
  matPoints = gedim.make_nd_matrix(points, (3, numPoints), np.double)
  values = forcing_term_ref_22(matPoints)
  for p in range(0, numPoints):
    if (matPoints[0, p] <= (1.0 + 1.0e-8)):
      values[p] = 0. 
  return values.ctypes.data
def ForcingTerm23(numPoints, points):
  matPoints = gedim.make_nd_matrix(points, (3, numPoints), np.double)
  values = forcing_term_ref_23(matPoints)
  for p in range(0, numPoints):
    if (matPoints[0, p] <= (1.0 + 1.0e-8)):
      values[p] = 0. 
  return values.ctypes.data

**Let us code the OFFLINE PHASE**

In [ ]:
order = 1

**CASE 1** - Creating Mesh Non-Conformed to the interface

In [ ]:
meshSize = 0.01
domain = { 'RectangleBase': 2.0, 'RectangleHeight': 1.0, 'VerticesBoundaryCondition': [1,1,1,1], 'EdgesBoundaryCondition': [1,1,1,1], 'DiscretizationType': 1, 'MeshCellsMaximumArea': meshSize }
[meshInfo, mesh] = gedim.CreateDomainRectangle(domain, lib)

**CASE 2** - Importing Mesh Conformed to the interface

In [ ]:
%%writefile ImportMesh.csv
InputFolderPath
./Meshes/Mesh6

In [ ]:
[meshInfo, mesh] = gedim.ImportDomainMesh2D(lib)

In [ ]:
gedim.PlotMesh(mesh)

### FEM space (the High Fidelity approximation)

In [ ]:
discreteSpace = { 'Order': order, 'Type': 1, 'BoundaryConditionsType': [1, 2] }
[problemData, dofs, strongs] = gedim.Discretize(discreteSpace, lib)

In [ ]:
gedim.PlotDofs(mesh, dofs, strongs)

### Assemble linear system exploting affinity
We define everything that is parameter independent:
$$\mathbb{A}_i,\ i \in \{0,\dots, q_a\} \quad \mathbf{f}_j,\ j \in \{0,\dots, q_f\}.$$
Moreover, we define the matrix $\mathbb{X}$ related to the scalar product of the problem at hand.
Finally, we create the parameter dependent variable:
$$θ^a_i(\boldsymbol{\mu}),\ i \in \{0,\dots, q_a\} \quad θ^f_j(\boldsymbol{\mu}),\ j \in \{0,\dots, q_f\}$$


In [ ]:
[stiffness1, stiffnessStrong1] = gedim.AssembleStiffnessMatrix(OmegaTilde1, problemData, lib)
[stiffness2_1, stiffnessStrong2_1] = gedim.AssembleAnisotropicStiffnessMatrix(OmegaTilde2_1, problemData, lib)
[stiffness2_2, stiffnessStrong2_2] = gedim.AssembleAnisotropicStiffnessMatrix(OmegaTilde2_2, problemData, lib)

forcingTerm11 = gedim.AssembleForcingTerm(ForcingTerm11, problemData, lib)
forcingTerm12 = gedim.AssembleForcingTerm(ForcingTerm12, problemData, lib)
forcingTerm21 = gedim.AssembleForcingTerm(ForcingTerm21, problemData, lib)
forcingTerm22 = gedim.AssembleForcingTerm(ForcingTerm22, problemData, lib)
forcingTerm23 = gedim.AssembleForcingTerm(ForcingTerm23, problemData, lib)

X = stiffness1 + stiffness2_1 + stiffness2_2

### define the problem
AQH = [stiffness1, stiffness2_1, stiffness2_2]
fQH = [forcingTerm11, forcingTerm12, forcingTerm21, forcingTerm22, forcingTerm23]

def thetaA(mu):
  return [1.0, 1.0 / mu[0], mu[0]]
def thetaF(mu):
  return [1.0, mu[0], mu[0], mu[0] * mu[0] * mu[0], mu[0] * mu[0] * (1.0 - mu[0])]

We will define some useful functions to perform computations:

In [ ]:
def normX(v, X):
	return np.sqrt(np.transpose(v) @ X @ v)

def ProjectSystem(AQH, fQH, B):
    AQN = []
    fQN = []
    for AH in AQH:
        AQN.append(np.copy(np.transpose(B) @ AH @ B))
    for fH in fQH:
        fQN.append(np.copy(np.transpose(B) @ fH))
    return [AQN, fQN]

def Solve_full_order(AQH, fQH, thetaA_mu, thetaF_mu):
    A = thetaA_mu[0] * AQH[0]
    f = thetaF_mu[0] * fQH[0]
    for i in range(1, len(AQH)):
        A += thetaA_mu[i] * AQH[i]
    for i in range(1, len(fQH)):
        f += thetaF_mu[i] * fQH[i]
    return gedim.LUSolver(A, f, lib)

def Solve_reduced_order(AQN, fQN, thetaA_mu, thetaF_mu):
    A = thetaA_mu[0] * AQN[0]
    f = thetaF_mu[0] * fQN[0]
    for i in range(1, len(AQN)):
        A += thetaA_mu[i] * AQN[i]
    for i in range(1, len(fQN)):
        f += thetaF_mu[i] * fQN[i]
    return np.linalg.solve(A, f)

We here define the finite parametric space $\mathcal P_{train}$, with random uniform distributed realization of $\boldsymbol \mu$.
The cardinality of $\mathcal P_{train}$ is set to $M=100$.

In [ ]:
### define the training set
M = 100
mu1_range = [1.0, 3.5]
P = np.array([mu1_range])

training_set = np.random.uniform(low=P[:, 0], high=P[:, 1], size=(M, P.shape[0]))

### POD



In [ ]:
def POD(AQH, fQH, X, N_max, tol):
    #### snapshot matrix creation
    snapshot_matrix = []

    for mu in training_set:
        snapshot = Solve_full_order(AQH, fQH, thetaA(mu), thetaF(mu))
        snapshot_matrix.append(np.copy(snapshot))
    
    snapshot_matrix = np.array(snapshot_matrix) 

    ### covariance matrix
    C = snapshot_matrix @ X @ np.transpose(snapshot_matrix) ## metti inner product
    L_e, VM_e = np.linalg.eig(C)
    eigenvalues = []
    eigenvectors = []

    for i in range(len(L_e)):
        eig_real = L_e[i].real
        eig_complex = L_e[i].imag
        assert np.isclose(eig_complex, 0.)
        eigenvalues.append(eig_real)
        eigenvectors.append(VM_e[i].real)

    total_energy = sum(eigenvalues)
    retained_energy_vector = np.cumsum(eigenvalues)
    relative_retained_energy = retained_energy_vector/total_energy

    if all(flag==False for flag in relative_retained_energy >= (1.0 - tol)):
        N = N_max
    else:
        N = np.argmax(relative_retained_energy >= (1.0 - tol)) + 1

    # Create the basis function matrix
    basis_functions = []
    for n in range(N):
        eigenvector =  eigenvectors[n]
        # basis = (1/np.sqrt(M))*np.transpose(snapshot_matrix)@eigenvector 
        basis = np.transpose(snapshot_matrix) @ eigenvector
        norm = normX(basis, X)
        # norm = np.sqrt(np.transpose(basis)@basis)
        basis /= norm
        basis_functions.append(np.copy(basis))

    return [N, np.transpose(np.array(basis_functions))]

### Offline Phase


In [ ]:
tol = 1.0e-7
N_max = 20

We perform now the POD as for comparison:

In [ ]:
### Compute POD
[N_POD, B_POD] = POD(AQH, fQH, X, N_max, tol)
[AQN_POD, fQN_POD] = ProjectSystem(AQH, fQH, B_POD)

### Online Phase

In the _online phase_ we can use all the pre-assembled quantities to generate a new solution for a new parameter. 



In [ ]:
mu_test = 3.5
MU_TILDE = mu_test

In [ ]:
def TestSingleParameter(AQH, fQH, AQN, fQN, B, mu):
    reduced_solution = Solve_reduced_order(AQN, fQN, thetaA(mu), thetaF(mu))
    full_solution = Solve_full_order(AQH, fQH, thetaA(mu), thetaF(mu))

    ###### plot #######
    proj_reduced_solution = B @ reduced_solution

    ### computing error
    error_function = full_solution - proj_reduced_solution
    error_norm_squared_component = np.transpose(error_function) @ X @ error_function
    abs_err_ROM = np.sqrt(abs(error_norm_squared_component))

    full_solution_norm_squared_component = np.transpose(full_solution) @ X @ full_solution
    rel_err_ROM = abs_err_ROM / np.sqrt(abs(full_solution_norm_squared_component))

    solutionStrong = np.zeros(problemData['NumberStrongs'])

    map_mu = mu[0]
    mappedMesh = Map(mesh, map_mu)
    mappedDofs = Map(dofs, map_mu)
    mappedStrongs = Map(strongs, map_mu)

    exact_solution_Dofs = exact_solution(mappedDofs)
    exact_solution_norm_squared_component = np.transpose(exact_solution_Dofs) @ X @ exact_solution_Dofs

    gedim.PlotSolution(mappedMesh, mappedDofs, mappedStrongs, exact_solution_Dofs, solutionStrong, "EXACT Solution")
    gedim.PlotSolution(mappedMesh, mappedDofs, mappedStrongs, full_solution, solutionStrong, "FULL ORDER Solution")
    gedim.PlotSolution(mappedMesh, mappedDofs, mappedStrongs, proj_reduced_solution, solutionStrong, "REDUCED ORDER Solution")
    
    abs_err_L2 = gedim.ComputeErrorL2(Poisson_exactSolution, full_solution, solutionStrong, lib)
    abs_err_H1 = gedim.ComputeErrorH1(Poisson_exactDerivativeSolution, full_solution, solutionStrong, lib)
    rel_err_L2 = abs_err_L2 / np.sqrt(abs(exact_solution_norm_squared_component))
    rel_err_H1 = abs_err_H1 / np.sqrt(abs(exact_solution_norm_squared_component))
  
    return [rel_err_ROM, abs_err_ROM, rel_err_L2, abs_err_L2, rel_err_H1, abs_err_H1]

In [ ]:
[rel_err_ROM, abs_err_ROM, rel_err_L2, abs_err_L2, rel_err_H1, abs_err_H1] = TestSingleParameter(AQH, fQH, AQN_POD, fQN_POD, B_POD, [mu_test])

print("DOFs","N","rel_err_ROM","abs_err_ROM","rel_err_L2","abs_err_L2","rel_err_H1","abs_err_H1")
print(problemData['NumberDOFs'],\
          N_POD,\
          '{:.4e}'.format(np.mean(rel_err_ROM)),\
          '{:.4e}'.format(np.mean(abs_err_ROM)),\
          '{:.4e}'.format(np.mean(rel_err_L2)),\
          '{:.4e}'.format(np.mean(abs_err_L2)),\
          '{:.4e}'.format(np.mean(rel_err_H1)),\
          '{:.4e}'.format(np.mean(abs_err_H1)))

We can now compute an error analysis over the parametric space, together with a _speed-up_ anaslysis.

The speed-up is an index that evaluated how many ROM solution I can obtain in the time of a FOM simulation.

In [ ]:
def Avg_error(AQH, fQH, AQN, fQN, B):
    ### compute avg error
    abs_err = []
    rel_err = []
    testing_set = np.random.uniform(low=P[:, 0], high=P[:, 1], size=(100, P.shape[0]))
    speed_up = []

    print("Computing error and speedup analysis...")

    for mu in testing_set:
        ##### full #####
        start_fom = time.time()
        full_solution = Solve_full_order(AQH, fQH, thetaA(mu), thetaF(mu))
        time_fom = time.time() - start_fom

        #### reduced #####
        start_rom = time.time()
        reduced_solution = Solve_reduced_order(AQN, fQN, thetaA(mu), thetaF(mu))
        time_rom = time.time() - start_rom

        speed_up.append(time_fom / time_rom)

        proj_reduced_solution = B @ reduced_solution

        ### computing error
        error_function = full_solution - proj_reduced_solution
        error_norm_squared_component = np.transpose(error_function) @ X @ error_function
        absolute_error = np.sqrt(abs(error_norm_squared_component))
        abs_err.append(absolute_error)

        full_solution_norm_squared_component = np.transpose(full_solution) @  X @ full_solution
        relative_error = absolute_error/np.sqrt(abs(full_solution_norm_squared_component))
        rel_err.append(relative_error)
    
    return [rel_err, abs_err, speed_up]

In [ ]:
[rel_err_POD, abs_err_POD, speed_up_POD] = Avg_error(AQH, fQH, AQN_POD, fQN_POD, B_POD)
print("- Average POD relative error = ", '{:.16e}'.format(np.mean(rel_err_POD)) )
print("- Average POD absolute error = ", '{:.16e}'.format(np.mean(abs_err_POD)) )
print("- Average POD speed_up       = ", '{:.16e}'.format(np.mean(speed_up_POD)))